In [3]:
import pandas as pd

def parse(path):
	g = open(path, 'rb')
	for l in g:
		yield eval(l)

def getDF(path):
	i = 0
	df = {}
	for d in parse(path):
		df[i] = d
		i += 1
	return pd.DataFrame.from_dict(df, orient='index')

In [3]:
def file_preprocess(rating_data, item_data):
    #read in data
    itemsFrame = getDF(item_data).loc[:,["asin", "description", "title", "price"]]
    print("item Num:")
    print(itemsFrame.shape[0]) 
    
    ratingsFrame = getDF(rating_data).loc[:,["reviewerID", "asin", "overall"]]    
    print("ratings Num:")
    print(ratingsFrame.shape[0])
        
    #modify userId to int
    print("Modifying users of ratingsFrame:")
    
    ratingsFrame.sort_values(by = 'reviewerID', ascending=True, inplace=True)
    
    userlist = ratingsFrame['reviewerID'].tolist()
    del ratingsFrame["reviewerID"]
    
    i = 0
    userId = userlist[0]
    for userNum in range(len(userlist)):
        if userNum % 10000 == 0:
            print("%.2f%%" %(100*userNum/ratingsFrame.shape[0]))
        if userlist[userNum] != userId:
            userId = userlist[userNum]
            i += 1
            userlist[userNum] = i
        else:
            userlist[userNum] = i
    ratingsFrame.insert(0, "reviewerID", userlist)
    print("ratingsFrame, users Modified")
    
    #modify itemId to int
    print("Modifying items of ratingsFrame:")
    
    itemsFrame.sort_values(by = 'asin', ascending=True, inplace=True)
    ratingsFrame.sort_values(by = 'asin', ascending=True, inplace=True)

    itemlist_rating = ratingsFrame['asin'].tolist()
    itemlist_info = itemsFrame['asin'].tolist()

    del ratingsFrame["asin"]
    del itemsFrame["asin"]
    
    for item in itemlist_rating:
        if item[0].isalpha():
            break
        elif item[0].isdigit() and len(item) < 10:
            itemlist_rating[itemlist_rating.index(item)] = '0' + item
            
    i = 0
    itemId = itemlist_rating[0]
    itemlist_info[itemlist_info.index(itemId)] = 0    
    for itemNum in range(len(itemlist_rating)):
        if itemNum % 5000 == 0:
            print("%.2f%%" %(100*itemNum/ratingsFrame.shape[0]))
        if itemlist_rating[itemNum] != itemId:
            itemId = itemlist_rating[itemNum]
            i += 1
            itemlist_rating[itemNum] = i
            itemlist_info[itemlist_info.index(itemId)] = i
        else:
            itemlist_rating[itemNum] = i
    print("ratingsFrame, items Modified")

    itemsFrame.insert(0, "asin", itemlist_info)  
    ratingsFrame.insert(1, "asin", itemlist_rating)
    itemsFrame_redundant = itemsFrame    
    itemsFrame = pd.DataFrame(columns=('asin', 'description', 'title', 'price'))
    
    print("Modifying itemsFrame:")
    i = 0
    for itemNum in range(len(itemlist_info)):
        if itemNum % 5000 == 0:
            print("%.2f%%" %(100*itemNum/itemsFrame_redundant.shape[0]))
        if type(itemlist_info[itemNum]) == int:
            itemsFrame.loc[i,:] = itemsFrame_redundant.iloc[itemNum,:]
            i += 1 
    print("itemsFrame Modified")
    
    return ratingsFrame, itemsFrame

In [4]:
# item_data = 'item_metadata/meta_Computers.json'
# rating_data = 'user_ratings/ratings_Computers.csv'
catagory = "Cell_Phones_and_Accessories"
item_data = 'item_metadata/meta_' + catagory + '.json'
rating_data = 'user_ratings/' + catagory + '_10.json'
item_data_processed = 'item_metadata/meta_' + catagory + '.csv'
rating_data_processed = 'user_ratings/' + catagory + '.csv'

In [5]:
ratingsFrame, itemsFrame = file_preprocess(rating_data, item_data)

item Num:
346793
ratings Num:
1854
Modifying users of ratingsFrame:
0.00%
ratingsFrame, users Modified
Modifying items of ratingsFrame:
0.00%
ratingsFrame, items Modified
Modifying itemsFrame:
0.00%
1.44%
2.88%
4.33%
5.77%
7.21%
8.65%
10.09%
11.53%
12.98%
14.42%
15.86%
17.30%
18.74%
20.18%
21.63%
23.07%
24.51%
25.95%
27.39%
28.84%
30.28%
31.72%
33.16%
34.60%
36.04%
37.49%
38.93%
40.37%
41.81%
43.25%
44.70%
46.14%
47.58%
49.02%
50.46%
51.90%
53.35%
54.79%
56.23%
57.67%
59.11%
60.55%
62.00%
63.44%
64.88%
66.32%
67.76%
69.21%
70.65%
72.09%
73.53%
74.97%
76.41%
77.86%
79.30%
80.74%
82.18%
83.62%
85.07%
86.51%
87.95%
89.39%
90.83%
92.27%
93.72%
95.16%
96.60%
98.04%
99.48%
itemsFrame Modified


In [8]:
users = ratingsFrame.sort_values(by="reviewerID").iloc[ratingsFrame.shape[0] - 1]["reviewerID"]
items = ratingsFrame.sort_values(by="asin").iloc[ratingsFrame.shape[0] - 1]["asin"]
ratings = ratingsFrame.shape[0]
print("users:")
print(users)
print("items:")
print(items)
print("ratings:")
print(ratings)
print("ratio:")
print(ratings/users/items)
ratingsFrame

users:
90.0
items:
115.0
ratings:
1854
ratio:
0.179130434783


,reviewerID,asin,overall
13,72,0,4.0
7,71,0,5.0
5,27,0,5.0
11,10,0,4.0
6,47,0,5.0
12,65,0,5.0
3,62,0,5.0
9,87,0,5.0
4,60,0,5.0
1,20,0,5.0


In [ ]:
ratingsFrame.to_csv(rating_data_processed, header = False, index = False)
itemsFrame.to_csv(item_data_processed, header = True, index = False)

In [1]:
ratingsFrame

NameError: name 'ratingsFrame' is not defined

In [ ]:
Cell_Phones_and_Accessories :
    users:
    90.0
    items:
    115.0
    ratings:
    1854
    ratio:
    0.179130434783
Grocery_and_Gourmet_Food : 
    users:
    1748.0
    items:
    548.0
    ratings:
    37348
    ratio:
    0.0389892932904
CDs_and_Vinyl:
    users:
    15591.0
    items:
    16183.0
    ratings:
    445412
    ratio:
    0.00176534221888
Movies_and_TV: 
    users:
    33325.0
    items:
    21900.0
    ratings:
    958986
    ratio:
    0.00131400795404


In [ ]:
Video_Games:
    users:
    2815.0
    items:
    2140.0
    ratings:
    52158
    ratio:
    0.00865822280507
Electronics:
    users:
    20246.0
    items:
    11588.0
    ratings:
    347393
    ratio:
    0.00148072136948
Kindle_Store:
    users:
    14355.0
    items:
    15884.0
    ratings:
    367478
    ratio:
    0.00161164085738
Books:
    users:
    158649.0
    items:
    128938.0
    ratings:
    4701968
    ratio:
    0.000229858940579

In [3]:
import pandas as pd
catagory = "Cell_Phones_and_Accessories"
item_data = '../Amazon-based/item_metadata/meta_' + catagory + '.csv'
rating_data = '../Amazon-based/user_ratings/' + catagory + '.csv'
ratingsFrame = pd.read_csv(rating_data, names = ["userID", "itemID", "rating"])
ratingsFrame.sort_values(by = 'userID', ascending=True, inplace=True)
itemsFrame = pd.read_csv(item_data)

In [66]:
import numpy as np
from scipy.sparse import *
row = np.array([0, 2, 2, 0, 1, 2])
col = np.array([0, 0, 1, 2, 2, 2])
data = np.array([1, 2, 3, 4, 5, 6])
a = csc_matrix((data, (row, col)), shape=(3, 3))